In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb 26 16:00:07 2021

In [ ]:
@author: Kuldeep Kumar
"""

In [ ]:
from matplotlib import pyplot
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

READ THE CSV FILES

In [ ]:
dataframe = pd.read_csv('happiness_score_dataset.csv')

In [ ]:
print("Look up into data",dataframe.head(10))

create the bar chart of top 10 Countries

In [ ]:
dataframe_top10 = dataframe.nlargest(10,['Happiness Score'])

In [ ]:
print(dataframe_top10)

ean of Hapiness score to get the thresold

In [ ]:
happyness_score_mean=dataframe['Happiness Score'].mean()
print(happyness_score_mean)

In [ ]:
dataframe=dataframe.drop(['Country','Region'],axis=1)

In [ ]:
print("Corelation between data",dataframe.corr())

In [ ]:
y=dataframe[["Happiness Score"]]
X=dataframe.drop(["Happiness Score"],axis=1)

In [ ]:
print("Lookup in to data",y.head(10))
#Null value for label
print("Null value in Label",y.isna().any())
print("Null value in X",X.isna().any())

=X.drop(["Country"],axis=1)

Scaling of the Data

In [ ]:
scaler1=StandardScaler()
X=pd.DataFrame(scaler1.fit_transform(X),columns=X.columns)
print("Look up into data after scaling for X",X.head())

In [ ]:
scaler2=StandardScaler()
y=pd.DataFrame(scaler2.fit_transform(y),columns=y.columns)
print("Look up into data after scaling for y",y.head())

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=22)
print("Shape of train data",X_train.shape)
print("Shape of test data ",X_test.shape)

Test options and evaluation metric

In [ ]:
num_folds = 10
seed = 7
scoring = 'neg_mean_squared_error'

Spot-Check Algorithms

In [ ]:
models = []
models.append(('LR', LinearRegression()))
models.append(('LASSO', Lasso()))
models.append(('EN', ElasticNet()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('SVR', SVR()))
models.append(('XGB', XGBRegressor()))

evaluate each model in turn

In [ ]:
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed,shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
#Out put is getting best for LinearRegression
# LR: -0.000000 (0.000000)
# LASSO: -1.017395 (0.450396)
# EN: -0.477101 (0.256295)
# KNN: -0.081843 (0.047432)
# CART: -0.010387 (0.011038)
# SVR: -0.064700 (0.093131)
# XGB: -0.005038 (0.004796)

Compare Algorithms

In [ ]:
fig = pyplot.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)
print(mean_squared_error(y_test, predictions))

In [ ]:
final_prediction_res=pd.DataFrame(predictions,columns=['Happyness_Score'])
final_prediction_res.to_excel("Final_Score.xlsx",index=False)